In [1]:
from taxcalc import *
import numpy as np
import pandas as pd
from pandas import DataFrame
import copy

In [2]:
# settings
# Change year here if running simulations for any other year
sim_year = 2017
agi_brackets = [(-1e14, 50000), (50000, 100000), (100000, 1e14)]
# list of dictionaries for each elasticity
elasticities = [
    {'lower': -0.5, 'middle': -0.5, 'upper': -0.5},
    {'lower': -1.0, 'middle': -1.0, 'upper': -1.0},
    {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}]

In [3]:
# lists for each output
base_list = list()
std_list = list()
charitable_list = list()
lower_mtr_list = list()
std_mtr_list = list()
std_charitable_list = list()
all_three_list = list()

In [4]:
# Reform dictionaries

# 2: raise standard deduction
std_reform = {
    sim_year: {
        '_STD': [[11000, 22000, 11000, 11000, 22000]]
    }
}

# 3: allow non-itemizers charitable deduction
charitable_reform = {
    sim_year: {
        '_STD_allow_charity_ded_nonitemizers': [True]
    }
}

# 4: lower top MTR to 35%
mtr_reform = {
    sim_year: {
        '_II_rt7': [0.35]
    }
}

In [5]:
# average charitable contributions by AGI level
contribs = pd.Series([342.96, 857.90, 1585.56])
recs = Records()
pol = Policy()

# blow-up contributions to 2013 levels
for year in range(2010, 2014):
    contribs *= recs.gfactors.factor_value('ATXPY', year)
contribs

0     389.598564
1     974.564405
2    1801.177687
dtype: float64

In [6]:
# intitalize calculator - used to assign charitable giving to non-itemizers
calc = Calculator(records=recs, policy=pol)
calc.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [7]:
# assign contribution amounts to non-itemizers based on AGI
calc.records.e19800 = np.where((calc.records.c00100 < 50000) & (calc.records.c04470 == 0),
                               contribs[0], calc.records.e19800)
calc.records.e19800 = np.where((calc.records.c00100 >= 50000) & (calc.records.c00100 < 100000) &
                               (calc.records.c04470 == 0),
                               contribs[1], calc.records.e19800)
calc.records.e19800 = np.where((calc.records.c00100 > 100000) & (calc.records.c04470 == 0),
                               contribs[2], calc.records.e19800)
charitable_array = copy.deepcopy(calc.records.e19800)

In [8]:
# generate output table
# Total charitable giving only consists of cash contributions. Non-cash can be easily added

def table(calc_table):
    
    dictionary = {'Revenue': [],
                  'Total Giving': [], 
                  'Total Giving: Itemizers': [], 
                  'Total Giving: Non-Itemizers': [], 
                  'Number of Itemizers': [], 
                  'Number of Non-Itemizers': []}
    # Loop through each AGI income bin to get values
    for item in agi_brackets:
        
        val = calc_table.records._combined[(calc_table.records.c00100 >= item[0])&
                                           (calc_table.records.c00100 < item[1])]
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                       (calc_table.records.c00100 < item[1])]
        dictionary['Revenue'].append(val.sum() / 1000000)
        
        val = calc_table.records.e19800[(calc_table.records.c00100 >= item[0])&
                                        (calc_table.records.c00100 < item[1])]
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                       (calc_table.records.c00100 < item[1])]
        dictionary['Total Giving'].append(float(val.sum()) / 1000000)
        
        val = calc_table.records.e19800[(calc_table.records.c00100 >= item[0])&
                                        (calc_table.records.c00100 < item[1])&
                                        (calc_table.records.c04470 > 0)]
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                       (calc_table.records.c00100 < item[1])&
                                       (calc_table.records.c04470 > 0)]
        dictionary['Total Giving: Itemizers'].append(float(val.sum()) / 1000000)
        
        val = calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                      (calc_table.records.c00100 < item[1])&
                                      (calc_table.records.c04470 > 0)]
        dictionary['Number of Itemizers'].append(float(val.sum()) / 1000000)

        val = calc_table.records.e19800[(calc_table.records.c00100 >= item[0])&
                                        (calc_table.records.c00100 < item[1])&
                                        (calc_table.records.c04470 == 0)]
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                       (calc_table.records.c00100 < item[1])&
                                       (calc_table.records.c04470 == 0)]
        dictionary['Total Giving: Non-Itemizers'].append(float(val.sum()) / 1000000)
        
        val = calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                      (calc_table.records.c00100 < item[1])&
                                      (calc_table.records.c04470 == 0)]
        dictionary['Number of Non-Itemizers'].append(float(val.sum()) / 1000000)
    # Add total rows
    for key in dictionary.keys():
        dictionary[key].append(sum(dictionary[key]))
        
    df = DataFrame.from_dict(dictionary)
    df.index = ['$0-50K', '$50K-100K', '$100K+', 'All']
    df = df.round(2)
    del dictionary
        
    return(df)

# Begin Simulations

In [10]:
# baseline function - no policy changes, but with elasticities
def base(elast):
    """
    elast: dictionary of elasticities
    returns DF to make the table we're interested in
    """
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    # Set calculator
    recs_base = Records()
    pol_base = Policy()
    beh_base = Behavior()
    calc_base = Calculator(records=recs_base, policy=pol_base, behavior=beh_base,
                          verbose=False)
    beh_base.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_base.records.e19800 = copy.deepcopy(charitable_array)
    
    # assuming we're doing the analysis for 2017
    calc_base.advance_to_year(sim_year)
    
    # create calculator with behavioral response
    calc_base_beh = Behavior.response(calc_noref, calc_base)
    
    # Create and append DF with information interested in
    base_list.append(table(calc_base_beh))

In [11]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
def std(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_std = Records()
    pol_std = Policy()
    pol_std.implement_reform(std_reform)
    beh_std = Behavior()
    calc_std = Calculator(records=recs_std, policy=pol_std, behavior=beh_std,
                          verbose=False)
    beh_std.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std.records.e19800 = copy.deepcopy(charitable_array)
    calc_std.advance_to_year(sim_year)
    
    # Create calculator with behavioral reform
    calc_std_beh = Behavior.response(calc_noref, calc_std)
    
    # create and append DF with information interested in
    std_list.append(table(calc_std_beh))

In [12]:
# ability for non-itemizers to itemize charitable deductions 
def charitable(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_charitable = Records()
    pol_charitable = Policy()
    pol_charitable.implement_reform(charitable_reform)
    beh_charitable = Behavior()
    calc_charitable = Calculator(records=recs_charitable, policy=pol_charitable, behavior=beh_charitable,
                                 verbose=False)
    beh_charitable.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_charitable.records.e19800 = copy.deepcopy(charitable_array)
    calc_charitable.advance_to_year(sim_year)
    
    # Create calculator with behaviroal reform
    calc_charitable_beh = Behavior.response(calc_noref, calc_charitable)
    
    # Create and append DF with information interested in
    charitable_list.append(table(calc_charitable_beh))

In [13]:
# decrease of the highest marginal tax rate to 35% 
def lower_mtr(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_mtr = Records()
    pol_mtr = Policy()
    pol_mtr.implement_reform(mtr_reform)
    beh_mtr = Behavior()
    calc_mtr = Calculator(records=recs_mtr, policy = pol_mtr, behavior=beh_mtr,
                          verbose=False)
    beh_mtr.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_mtr.records.e19800 = copy.deepcopy(charitable_array)
    calc_mtr.advance_to_year(sim_year)
    
    # Calculator with behavioral reform
    calc_mtr_beh = Behavior.response(calc_noref, calc_mtr)
    
    # Create and append DF with information interested
    lower_mtr_list.append(table(calc_mtr_beh))

In [14]:
# standard deduction change and the marginal rate decrease
def std_mtr(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_std_mtr = Records()
    pol_std_mtr = Policy()
    pol_std_mtr.implement_reform(std_reform)
    pol_std_mtr.implement_reform(mtr_reform)
    beh_std_mtr = Behavior()
    calc_std_mtr = Calculator(records=recs_std_mtr, policy=pol_std_mtr, behavior=beh_std_mtr,
                              verbose=False)
    calc_std_mtr.records.e19800 = copy.deepcopy(charitable_array)
    beh_std_mtr.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std_mtr.advance_to_year(sim_year)
    
    # Calculator with behavioral reform
    calc_std_mtr_beh = Behavior.response(calc_noref, calc_std_mtr)
    
    # Create and append df with information interested in
    std_mtr_list.append(table(calc_std_mtr_beh))

In [15]:
# standard deduction change AND the non-itemizer deduction change
def std_charitable(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_std_charitable = Records()
    pol_std_charitable = Policy()
    pol_std_charitable.implement_reform(std_reform)
    pol_std_charitable.implement_reform(charitable_reform)
    beh_std_charitable = Behavior()
    calc_std_charitable = Calculator(records=recs_std_charitable, policy=pol_std_charitable,
                                     behavior=beh_std_charitable, verbose=False)
    calc_std_charitable.records.e19800 = copy.deepcopy(charitable_array)
    beh_std_charitable.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std_charitable.advance_to_year(sim_year)
    
    # Calculator with behavioral response
    calc_std_charitable_beh = Behavior.response(calc_noref, calc_std_charitable)
    
    # Create and append DF with information
    std_charitable_list.append(table(calc_std_charitable_beh))

In [16]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
def all_three(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_three = Records()
    pol_three = Policy()
    pol_three.implement_reform(std_reform)
    pol_three.implement_reform(charitable_reform)
    pol_three.implement_reform(mtr_reform)
    beh_three = Behavior()
    calc_three = Calculator(records=recs_three, policy=pol_three, behavior=beh_three,
                            verbose=False)
    calc_three.records.e19800 = copy.deepcopy(charitable_array)
    beh_three.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_three.advance_to_year(sim_year)
    
    # Calculator with behavior reform
    calc_three_beh = Behavior.response(calc_noref, calc_three)
    
    # Create and append DF with information
    all_three_list.append(table(calc_three_beh))

In [17]:
# loop through each branch of elasticites, calling each function
for item in elasticities:
    base(item)
    std(item)
    charitable(item)
    lower_mtr(item)
    std_mtr(item)
    std_charitable(item)
    all_three(item)

taxcalc/behavior.py:306: RuntimeWarning: invalid value encountered in divide
  delta_income * calc.records.e00200 / agi_m_ided,
taxcalc/behavior.py:310: RuntimeWarning: invalid value encountered in divide
  delta_income * other_income / agi_m_ided,
taxcalc/behavior.py:313: RuntimeWarning: invalid value encountered in divide
  delta_income * ided / agi_m_ided,


# Output (In Millions)

Each table is in the form of a DataFrame and is ready to be exported if need be

# baseline function

In [18]:
# baseline function
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
base_list[0]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,7.45,100.64,206210.44,55624.30,10646.29,44978.01
$50K-100K,13.99,18.41,477849.84,48952.55,31849.59,17102.97
$100K+,23.85,6.08,2108967.48,140300.51,129942.15,10358.36
All,45.29,125.13,2793027.75,244877.37,172438.03,72439.34


In [19]:
# baseline function
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
base_list[1]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,7.45,100.64,206210.44,55624.30,10646.29,44978.01
$50K-100K,13.99,18.41,477849.84,48952.55,31849.59,17102.97
$100K+,23.85,6.08,2108967.48,140300.51,129942.15,10358.36
All,45.29,125.13,2793027.75,244877.37,172438.03,72439.34


In [20]:
# baseline function
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
base_list[2]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,7.45,100.64,206210.44,55624.30,10646.29,44978.01
$50K-100K,13.99,18.41,477849.84,48952.55,31849.59,17102.97
$100K+,23.85,6.08,2108967.48,140300.51,129942.15,10358.36
All,45.29,125.13,2793027.75,244877.37,172438.03,72439.34


# increase value of the standard deduction

In [21]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
std_list[0]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,3.57,104.52,184714.26,55331.69,5597.96,49733.73
$50K-100K,8.11,24.29,453012.48,47865.26,21286.80,26578.46
$100K+,17.73,12.20,2091078.69,138338.39,118003.67,20334.72
All,29.41,141.01,2728805.43,241535.34,144888.43,96646.92


In [22]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
std_list[1]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,3.57,104.52,184714.26,55624.30,5597.96,50026.34
$50K-100K,8.11,24.29,453012.48,48952.55,21286.80,27665.76
$100K+,17.73,12.20,2091078.69,140300.51,118003.67,22296.84
All,29.41,141.01,2728805.43,244877.37,144888.43,99988.94


In [23]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
std_list[2]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,3.57,104.52,184714.26,54315.74,5597.96,48717.78
$50K-100K,8.11,24.29,453012.48,45712.42,21286.80,24425.62
$100K+,17.73,12.20,2091078.69,135662.07,118003.67,17658.39
All,29.41,141.01,2728805.43,235690.23,144888.43,90801.80


# ability for non-itemizers to itemize charitable deductions 

In [24]:
# ability for non-itemizers to itemize charitable deductions 
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
charitable_list[0]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,6.20,101.89,203486.26,56717.69,5333.49,51384.20
$50K-100K,11.87,20.53,473426.92,50498.43,19284.69,31213.75
$100K+,22.29,7.64,2104540.29,141428.53,114463.59,26964.95
All,40.35,130.07,2781453.47,248644.66,139081.77,109562.89


In [25]:
# ability for non-itemizers to itemize charitable deductions
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
charitable_list[1]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,6.20,101.89,203634.24,55624.30,5333.49,50290.81
$50K-100K,11.87,20.53,473747.78,48952.55,19284.69,29667.87
$100K+,22.29,7.64,2104892.58,140300.51,114463.59,25836.92
All,40.35,130.07,2782274.60,244877.37,139081.77,105795.60


In [26]:
# ability for non-itemizers to itemize charitable deductions
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
charitable_list[2]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,6.20,101.89,202977.64,60513.92,5333.49,55180.43
$50K-100K,11.87,20.53,472798.57,53559.28,19284.69,34274.59
$100K+,22.29,7.64,2104116.83,142967.16,114463.59,28503.57
All,40.35,130.07,2779893.04,257040.36,139081.77,117958.59


# decrease of the highest marginal tax rate to 35% # Lower MTR Results

In [27]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
lower_mtr_list[0]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,7.45,100.64,206210.44,55624.30,10646.29,44978.01
$50K-100K,13.99,18.41,477849.84,48952.55,31849.59,17102.97
$100K+,23.85,6.08,2084483.73,139562.00,129209.21,10352.79
All,45.29,125.13,2768544.01,244138.85,171705.08,72433.77


In [28]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
lower_mtr_list[1]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,7.45,100.64,206210.44,55624.30,10646.29,44978.01
$50K-100K,13.99,18.41,477849.84,48952.55,31849.59,17102.97
$100K+,23.85,6.08,2084180.59,140300.51,129953.67,10346.84
All,45.29,125.13,2768240.87,244877.37,172449.55,72427.82


In [29]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
lower_mtr_list[2]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,7.45,100.64,206210.44,55624.30,10646.29,44978.01
$50K-100K,13.99,18.41,477849.84,48952.55,31849.59,17102.97
$100K+,23.85,6.08,2084897.78,138554.66,128194.21,10360.45
All,45.29,125.13,2768958.06,243131.52,170690.09,72441.43


# standard deduction change and the marginal rate decrease

In [30]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
std_mtr_list[0]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,3.57,104.52,184714.26,55331.69,5597.96,49733.73
$50K-100K,8.11,24.29,453012.48,47865.26,21286.80,26578.46
$100K+,17.74,12.19,2066646.04,137605.81,117271.46,20334.35
All,29.41,141.00,2704372.78,240802.76,144156.21,96646.55


In [31]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
std_mtr_list[1]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,3.57,104.52,184714.26,55624.30,5597.96,50026.34
$50K-100K,8.11,24.29,453012.48,48952.55,21286.80,27665.76
$100K+,17.74,12.19,2066344.85,140300.51,118019.07,22281.44
All,29.42,141.00,2704071.59,244877.37,144903.83,99973.54


In [32]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
std_mtr_list[2]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,3.57,104.52,184714.26,54315.74,5597.96,48717.78
$50K-100K,8.11,24.29,453012.48,45712.42,21286.80,24425.62
$100K+,17.74,12.19,2067057.13,133930.24,116246.40,17683.84
All,29.41,141.01,2704783.87,233958.40,143131.16,90827.25


# standard deduction change AND the non-itemizer deduction change

In [33]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
std_charitable_list[0]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.79,105.30,182216.23,56327.77,2135.44,54192.33
$50K-100K,6.43,25.97,446558.06,50304.78,10525.83,39778.94
$100K+,15.70,14.23,2083103.27,140364.01,99556.59,40807.42
All,24.92,145.50,2711877.55,246996.56,112217.87,134778.70


In [34]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
std_charitable_list[1]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.79,105.30,182321.55,55624.30,2135.44,53488.86
$50K-100K,6.43,25.97,446844.28,48952.55,10525.83,38426.72
$100K+,15.70,14.23,2083338.18,140300.51,99556.59,40743.92
All,24.92,145.50,2712504.01,244877.37,112217.87,132659.50


In [35]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
std_charitable_list[2]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.79,105.30,181856.94,58770.23,2135.44,56634.79
$50K-100K,6.43,25.97,446002.06,52982.18,10525.83,42456.35
$100K+,15.70,14.23,2082922.06,140450.63,99556.59,40894.04
All,24.92,145.50,2710781.06,252203.04,112217.87,139985.17


# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)

In [36]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
all_three_list[0]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.79,105.30,182216.23,56327.77,2135.44,54192.33
$50K-100K,6.43,25.97,446558.06,50304.78,10525.83,39778.94
$100K+,15.71,14.23,2058790.36,139610.92,99105.48,40505.44
All,24.92,145.50,2687564.65,246243.47,111766.76,134476.72


In [37]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
all_three_list[1]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.79,105.30,182321.55,55624.30,2135.44,53488.86
$50K-100K,6.43,25.97,446844.28,48952.55,10525.83,38426.72
$100K+,15.71,14.23,2058710.78,140300.51,99765.43,40535.09
All,24.92,145.49,2687876.60,244877.37,112426.70,132450.67


In [38]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
all_three_list[2]

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.79,105.30,181856.94,58770.23,2135.44,56634.79
$50K-100K,6.43,25.97,446002.06,52982.18,10525.83,42456.35
$100K+,15.71,14.23,2059031.43,138670.33,98207.57,40462.76
All,24.92,145.50,2686890.43,250422.74,110868.85,139553.89
